In [630]:
import bibtexparser as bp
import pandas as pd
import numpy as np
import yaml as yml

# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dirConfig = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos'
configFileName = 'config_analisa_bibtex.yml'

with open(dirConfig + '\\' + configFileName) as f:
    configFile = yml.load(f, Loader=yml.loader.SafeLoader)


In [631]:
# ANALISA BIBTEX
listaArquivos = ( \
	configFile['FILE_ACM'], \
	configFile['FILE_IEE'], \
	configFile['FILE_SD' ]
	)

formatFile = configFile['FORMATO']

dirInput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\input'
dirOutput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\output'

mergedFilesCSV = 'ALL_ARTICLES.csv'
mergedFilesJSON = 'ALL_ARTICLES.json'
mergedFilesYAML = 'ALL_ARTICLES.yml'

def bibtexToDict(arqv):
	arquivo = arqv
	with open(dirInput + '\\' + arquivo, encoding='utf8') as f:
		bib_database = bp.load(f)		
	return bib_database.entries_dict.values() # retorna lista de dicionarios
	
# UNION DATAFRAMES AND YML
listaDf = []
listaYml =[]
for arqv in listaArquivos:
	sourceArticles = bibtexToDict(arqv)
	listaDf.append(pd.DataFrame(sourceArticles))
	listaYml.append(yml.dump(list(sourceArticles)))

unionDf = pd.concat(listaDf)
unionYml = ''.join(listaYml)

In [632]:
# EXPORT
if formatFile == 'CSV':
	unionDf.to_csv(dirOutput + '\\' + mergedFilesCSV, index=False, encoding='utf-8')
elif formatFile == 'JSON':
	unionDf.to_json(dirOutput + '\\' + mergedFilesJSON, orient = 'records')
elif formatFile == 'YAML':
	with open(dirOutput + '\\' + mergedFilesYAML, mode='w',encoding='utf8') as f:
		f.write(unionYml)
else:
	print('Formato não disponivel.')


# Article Impact

In [633]:
import pandas as pd
import numpy as np


dir_iput_aula3 = r'C:\Users\Saulo\Documents\GitHub\MBA_PyForDE\script_analisa_bibtex\input'
dir_output_aula3 = r'C:\Users\Saulo\Documents\GitHub\MBA_PyForDE\script_analisa_bibtex\output'

file_scimagojr = 'scimagojr 2020.csv'
file_jcs = 'jcs_2020.csv'
file_article_impact = 'ALL_Article_Impact.csv'

df_scimagojr = pd.read_csv(dir_iput_aula3 + '\\' + file_scimagojr, delimiter=';', quotechar='"', header=0)
df_jcs = pd.read_csv(dir_iput_aula3 + '\\' + file_jcs, delimiter=';', quotechar='"', header=0)

# TRATAMENTO
# Remove colunas vazias no arquivo
df_scimagojr.drop(df_scimagojr.columns[df_scimagojr.columns.str.contains('Unnamed:') == True], axis=1, inplace=True)
df_jcs.drop(df_jcs.columns[df_jcs.columns.str.contains('Unnamed:') == True], axis=1, inplace=True)

df_jcs.rename(columns={'Full Journal Title':'Title'}, inplace=True)

df_scimagojr['Title'] = df_scimagojr['Title'].str.upper()
df_jcs['Title'] = df_jcs['Title'].str.upper()

# UNION
df_scimagojr_jcs = pd.concat((df_scimagojr,df_jcs))

# TRATAMENTO UNION
df_scimagojr_jcs['Rank'] = np.arange(0,len(df_scimagojr_jcs))
df_scimagojr_jcs.drop_duplicates() # Deduplicação dos dados

# EXPORT
# df_scimagojr_jcs.to_csv(dir_output_aula3 + '\\' + file_article_impact, index=False, encoding='utf-8')
# df_scimagojr_jcs.shape


c:\Users\Saulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),...,Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Total Cites,Journal Impact Factor,Eigenfactor Score
0,0,2.877300e+04,CA-A CANCER JOURNAL FOR CLINICIANS,journal,"15424863, 00079235","62,937",Q1,168.0,47.0,119.0,...,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),NaN,NaN,NaN
1,1,1.943400e+04,MMWR RECOMMENDATIONS AND REPORTS,journal,"10575987, 15458601","40,949",Q1,143.0,10.0,9.0,...,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,NaN,NaN,NaN
2,2,2.031500e+04,NATURE REVIEWS MOLECULAR CELL BIOLOGY,journal,"14710072, 14710080","37,461",Q1,431.0,115.0,338.0,...,"32,83","73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),NaN,NaN,NaN
3,3,2.943100e+04,QUARTERLY JOURNAL OF ECONOMICS,journal,"00335533, 15314650","34,573",Q1,259.0,40.0,110.0,...,"16,00","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1),NaN,NaN,NaN
4,4,2.110081e+10,NATURE REVIEWS MATERIALS,journal,20588437,"32,011",Q1,108.0,92.0,264.0,...,"32,15","115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13005,45957,NaN,ORNITHOLOGY RESEARCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Not Available,0.00000
13006,45958,NaN,PHYSICAL AND ENGINEERING SCIENCES IN MEDICINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184,Not Available,0.00002
13007,45959,NaN,RSC MEDICINAL CHEMISTRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,Not Available,0.00000
13008,45960,NaN,STUDIES IN PSYCHOLOGY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,Not Available,0.00000


# Article Impact and Bibtex

In [634]:
# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dir_config = r'c:\Users\Saulo\Documents\GitHub\MBA_PyForDE\script_analisa_bibtex\config'
config_file_name = 'config_article_impact.yml'
file_bibtex_impact = 'bibtex_impact'



with open(dir_config + '\\' + config_file_name) as f:
    config_file = yml.load(f, Loader=yml.loader.SafeLoader)

filter_list = ( config_file['TITLE'],\
	config_file['KEYWORDS'],\
	config_file['YEAR'],\
	config_file['TYPE_PUBLICATION'],\
	config_file['DOI'],\
	config_file['JCR_VALUE'],\
	config_file['SCIMAGO_VALUE'],\
	config_file['FORMATO']
	)
print(filter_list)



('Big', 'Data', None, None, None, None, 10.219, 'CSV')


Tratamento

In [635]:

df_bibtex = unionDf
df_bibtex.columns = list(map(lambda col: col.upper(), df_bibtex.columns))
df_scimagojr_jcs.columns = list(map(lambda col: col.upper(), df_scimagojr_jcs.columns))

df_scimagojr_jcs.rename(columns ={'JOURNAL IMPACT FACTOR':'JCR_VALUE'}, inplace=True)
df_scimagojr_jcs.rename(columns ={'SJR':'SCIMAGO_VALUE'}, inplace=True)

dict_treat_data = {'': np.nan, None: np.nan, 'Not Available': np.nan}
df_scimagojr_jcs.SCIMAGO_VALUE.replace(to_replace=dict_treat_data, inplace=True)
df_scimagojr_jcs.JCR_VALUE.replace(to_replace=dict_treat_data, inplace=True)

# np.nan já é float
df_scimagojr_jcs.JCR_VALUE[df_scimagojr_jcs.JCR_VALUE.notnull()] = [x.replace(',','.') for x in df_scimagojr_jcs.JCR_VALUE if type(x) != float]
df_scimagojr_jcs.SCIMAGO_VALUE[df_scimagojr_jcs.SCIMAGO_VALUE.notnull()] = [x.replace(',','.') for x in df_scimagojr_jcs.SCIMAGO_VALUE if type(x) != float]
df_scimagojr_jcs.JCR_VALUE = pd.to_numeric(df_scimagojr_jcs.JCR_VALUE)
df_scimagojr_jcs.SCIMAGO_VALUE = pd.to_numeric(df_scimagojr_jcs.SCIMAGO_VALUE)


C:\Users\Saulo\AppData\Local\Temp/ipykernel_25096/122150049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scimagojr_jcs.JCR_VALUE[df_scimagojr_jcs.JCR_VALUE.notnull()] = [x.replace(',','.') for x in df_scimagojr_jcs.JCR_VALUE if type(x) != float]
C:\Users\Saulo\AppData\Local\Temp/ipykernel_25096/122150049.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scimagojr_jcs.SCIMAGO_VALUE[df_scimagojr_jcs.SCIMAGO_VALUE.notnull()] = [x.replace(',','.') for x in df_scimagojr_jcs.SCIMAGO_VALUE if type(x) != float]


Gerando massa de dados

In [636]:
# Subistitui df_scimagojr_jcs com tilulos aleatorios do bibtex
np.random.seed(0)
title_number = 100
title_sample = list(unionDf.TITLE.iloc[np.random.randint(0,len(unionDf),title_number)])

for indice, title in list(zip(np.arange(0,title_number),title_sample)):
	df_scimagojr_jcs.TITLE.loc[indice] = title

c:\Users\Saulo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\Saulo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\Saulo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

# Join e Filtro

In [637]:
# JOIN
df_bibtex_impact = df_bibtex.merge(df_scimagojr_jcs, how='left', on='TITLE', suffixes=['_1','_2'])

df_filtrado = df_bibtex_impact.loc[
	(df_bibtex_impact.TITLE.str.contains("{}".format(config_file['TITLE']), case=False)) |
	(df_bibtex_impact.KEYWORDS.str.contains("{}".format(config_file['KEYWORDS']), case=False)) |
	(df_bibtex_impact.ABSTRACT.str.contains("{}".format(config_file['ABSTRACT']), case=False)) |
	(df_bibtex_impact.YEAR.str.contains("{}".format(config_file['YEAR']), case=False)) |
	(df_bibtex_impact.TYPE.str.contains("{}".format(config_file['TYPE_PUBLICATION']), case=False)) |
	(df_bibtex_impact.DOI.str.contains("{}".format(config_file['DOI']), case=False))
	#(df_bibtex_impact['JCR_VALUE'] == config_file['JCR_VALUE']) |
	#(df_bibtex_impact['SCIMAGO_VALUE'] == config_file['SCIMAGO_VALUE']) 
	]

df_filtro_final = df_filtrado.loc[(df_bibtex_impact['JCR_VALUE'] == config_file['JCR_VALUE']) |
(df_bibtex_impact['SCIMAGO_VALUE'] == config_file['SCIMAGO_VALUE'])]


In [638]:
df_filtro_final.loc[df_filtro_final['SCIMAGO_VALUE'].notnull()][['TITLE','KEYWORDS','JCR_VALUE','SCIMAGO_VALUE']]

,TITLE,KEYWORDS,JCR_VALUE,SCIMAGO_VALUE
1,Towards a Data Quality Assessment in Big Data,"Big Data, Data Quality, Quality Models, Data Q...",NaN,10.219


### EXPORT

In [639]:
# EXPORT
if config_file['FORMATO'] == 'CSV':
	df_filtro_final.to_csv(dir_output_aula3 + '\\' + file_bibtex_impact + '.csv', index=False, encoding='utf-8')
elif config_file['FORMATO'] == 'JSON':
	df_filtro_final.to_json(dir_output_aula3 + '\\' + file_bibtex_impact + '.json', orient = 'records')
elif config_file['FORMATO'] == 'YAML':
	yaml = yml.dump(df_filtro_final.to_dict(orient='records'))
	with open(dir_output_aula3 + '\\' + file_bibtex_impact + '.yml', mode='w',encoding='utf8') as f:
		f.write(yaml)
else:
	print('Formato não disponivel.')